## **Bioprospecting analysis 1.0**

#### This notebook integrates the metagenomic bioprospecting analysis  1.0.  
#### The tasks are organized in the following sections:  
#### [**0. Set env**](#section0)
#### [**1. Identify of BGCs**](#section1)
#### [**2. Compute BGC class abundances**](#section2)
#### [**3. Taxonomic annotation of BGCs**](#section3)
#### [**4. Map BGCs onto reference Gene Cluster Families (GCFs)**](#section4)
#### [**5. Compute BGCs diversity, functionally annotate, and assess novelty**]((#section5)

#### Dependencies to run this notebook (outside the tools we provide):  
#### [aws cli](https://aws.amazon.com/cli/)  
#### [bash and R kernels](https://evodify.com/python-r-bash-jupyter-notebook/)  
#### [Docker](https://www.docker.com/)
#### [RSQLite R library](https://cran.r-project.org/web/packages/RSQLite/index.html)
#### [tidyverse R library](https://www.tidyverse.org/)
#### [seqtk](https://github.com/lh3/seqtk)


<a id='section0'></a>
**0. Set env**

In [2]:
%load_ext rpy2.ipython
%set_env WORKDIR=workdir
%set_env REPO=/home/epereira/workspace/dev/new_atlantis/repos/bioprospecting
%set_env seqtk=/nfs/bin/seqtk/seqtk

env: WORKDIR=workdir
env: REPO=/home/epereira/workspace/dev/new_atlantis/repos/bioprospecting
env: seqtk=/nfs/bin/seqtk/seqtk


In [4]:
%%bash

mkdir -p ${WORKDIR}/data/sola
mkdir -p ${WORKDIR}/outputs/antismash

<a id='section1'></a>
**1. Identification of BGCs**

We will be using the [SOLA metagenomic dataset](https://www.nature.com/articles/s41396-018-0158-1), already assembled with [VEBA](https://github.com/jolespin/veba).
Let’s first get the data.

In [12]:
%%bash

# aws s3 cp s3://newatlantis-case-studies/SOLA-samples/ ${WORKDIR}/data/sola --recursive

This dataset contains the assembled scaffolds (\*.fasta) and the mapping files (\*.bam).

In [2]:
%%bash

ls ${WORKDIR}/data/sola/ERR*/output | head -12

workdir/data/sola/ERR2604071/output:
featurecounts.tsv.gz
mapped.sorted.bam
mapped.sorted.bam.bai
scaffolds.fasta
scaffolds.fasta.1.bt2
scaffolds.fasta.2.bt2
scaffolds.fasta.3.bt2
scaffolds.fasta.4.bt2
scaffolds.fasta.rev.1.bt2
scaffolds.fasta.rev.2.bt2
scaffolds.fasta.saf


Now that we have the data, let's run [antisMASH](https://github.com/antismash/antismash) to identify the BGC sequences.  
For this we will be using our wrap script [run_antismash](https://github.com/pereiramemo/bioprospecting/blob/main/run_scripts/run_antismash.sh), which runs a containerized version 6.0.0 of antiSMASH.  
Note that there is version 7.0.0 available, but for compatibility purposes in downstream analysis, we'll use this version for now.
Since we are using a wrap script to run a containerized version of antiSMASH, we have to use the fist two positional parameters as the input and output folders, respectively.  
To see the help we run:

In [3]:
%%bash

"${REPO}"/run_scripts/run_antismash.sh . . --help


########### antiSMASH 6.0.0 #############

usage: antismash [-h] [options ..] sequence


arguments:
  SEQUENCE  GenBank/EMBL/FASTA file(s) containing DNA.

--------
Options
--------
-h, --help              Show this help text.
--help-showall          Show full lists of arguments on this help text.
-c CPUS, --cpus CPUS    How many CPUs to use in parallel. (default: 48)

Basic analysis options:

  --taxon {bacteria,fungi}
                        Taxonomic classification of input sequence. (default:
                        bacteria)

Additional analysis:

  --fullhmmer           Run a whole-genome HMMer analysis.
  --cassis              Motif based prediction of SM gene cluster regions.
  --clusterhmmer        Run a cluster-limited HMMer analysis.
  --tigrfam             Annotate clusters using TIGRFam profiles.
  --smcog-trees         Generate phylogenetic trees of sec. met. cluster
                        orthologous groups.
  --tta-threshold TTA_THRESHOLD
                        Lowes

Let's run antiSMASH on the SOLA dataset.

In [3]:
%%bash

SCAFFOLDS=$(ls ${WORKDIR}/data/sola/ERR*/output/scaffolds.fasta)
for SCAFFOLD in ${SCAFFOLDS}; do

  SAMPLE_NAME=$(echo "${SCAFFOLD}" | sed "s/.*\(ERR[0-9]\+\)\/output.*/\1/");
  OUTPUT_DIR="${WORKDIR}/outputs/antismash/${SAMPLE_NAME}";
  echo "${SAMPLE_NAME}";
 
  "${REPO}"/run_scripts/run_antismash.sh "${SCAFFOLD}" "${OUTPUT_DIR}" \
  --cpus 40 \
  --genefinding-tool prodigal-m \
  --taxon bacteria \
  --allow-long-headers \
  --minlength 5000;

done


ERR2604071
ERR2604073
ERR2604074
ERR2604075
ERR2604076
ERR2604077
ERR2604078
ERR2604079
ERR2604080
ERR2604081
ERR2604082
ERR2604083
ERR2604084
ERR2604085
ERR2604086
ERR2604087
ERR2604088
ERR2604090
ERR2604091
ERR2604092
ERR2604093
ERR2604094
ERR2604095
ERR2604096
ERR2604097
ERR2604098
ERR2604099
ERR2604100
ERR2604101
ERR2604102
ERR2604103
ERR2604104
ERR2604105
ERR2604106
ERR2604107
ERR2604108
ERR2604109
ERR2604110


The annotated BGC sequences can be found in `${WORKDIR}/outputs/antismash/`

In [4]:
%%bash

ls ${WORKDIR}/outputs/antismash/

ERR2604071
ERR2604073
ERR2604074
ERR2604075
ERR2604076
ERR2604077
ERR2604078
ERR2604079
ERR2604080
ERR2604081
ERR2604082
ERR2604083
ERR2604084
ERR2604085
ERR2604086
ERR2604087
ERR2604088
ERR2604090
ERR2604091
ERR2604092
ERR2604093
ERR2604094
ERR2604095
ERR2604096
ERR2604097
ERR2604098
ERR2604099
ERR2604100
ERR2604101
ERR2604102
ERR2604103
ERR2604104
ERR2604105
ERR2604106
ERR2604107
ERR2604108
ERR2604109
ERR2604110


<a id='section2'></a>
**2. Compute BGC class abundances**

To do this, let's first create a directory where we will put the concatenated BGC gbk and the bam mapping files.  

In [5]:
%%bash
# mkdir ${WORKDIR}/outputs/bgc_abund

ls -d ${WORKDIR}/outputs/antismash/ERR* | \
while read LINE; do
  SAMPLE=$(basename "${LINE}")
  cat "${LINE}"/scaffolds/"${SAMPLE}"*.region*.gbk > "${WORKDIR}"/outputs/bgc_abund/"${SAMPLE}".gbk
done


Now we can run our custom scripts [get_cov.py](https://github.com/pereiramemo/bioprospecting/blob/main/aux_scripts/get_cov.py).  

In [6]:
%%bash

ls "${WORKDIR}"/outputs/bgc_abund/*.gbk | \
while read LINE; do

  SAMPLE=$(basename "${LINE}" .gbk)
  
  "${REPO}"/aux_scripts/get_cov.py \
  --input_gbk "${WORKDIR}"/outputs/bgc_abund/"${SAMPLE}".gbk \
  --input_bam "${WORKDIR}"/data/sola/"${SAMPLE}"/output/mapped.sorted.bam \
  --sample_name "${SAMPLE}" \
  --output_tsv "${WORKDIR}"/outputs/bgc_abund/"${SAMPLE}".tsv

done


In [7]:
%%bash

head "${WORKDIR}"/outputs/bgc_abund/ERR2604071.tsv

ERR2604071__k119_144067	phosphonate	True	0	6932	1.0418349682631276	ERR2604071	
ERR2604071__k119_154594	betalactone	True	0	11920	1.014010067114094	ERR2604071	
ERR2604071__k119_29665	resorcinol	True	0	6522	1.0111928856179087	ERR2604071	
ERR2604071__k119_29813	terpene	True	0	12578	1.02146605183654	ERR2604071	
ERR2604071__k119_50764	terpene	True	0	6293	1.034164945177181	ERR2604071	
ERR2604071__k119_63513	terpene	True	0	10997	1.0230972083295444	ERR2604071	
ERR2604071__k119_65188	terpene	True	0	13915	1.0409629895795904	ERR2604071	


We obtain a tsv file with the following fields: `scafflod id, BGC class, edge, start, end, coverage, sample`

Let's concatenate all these files into one tsv table (sample, bgc class, coverage):

In [12]:
%%bash

cat "${WORKDIR}"/outputs/bgc_abund/ERR*.tsv | awk -v OFS="\t" '{print $7,$2,$6}' > "${WORKDIR}"/outputs/bgc_abund/bgc_abund.tsv
head "${WORKDIR}"/outputs/bgc_abund/bgc_abund.tsv

ERR2604071	phosphonate	1.0418349682631276
ERR2604071	betalactone	1.014010067114094
ERR2604071	resorcinol	1.0111928856179087
ERR2604071	terpene	1.02146605183654
ERR2604071	terpene	1.034164945177181
ERR2604071	terpene	1.0230972083295444
ERR2604071	terpene	1.0409629895795904
ERR2604073	terpene	1.0692621095518333
ERR2604073	betalactone	1.0658359680119025
ERR2604073	hserlactone	1.0689745607133492


<a id='section3'></a>
**3. Taxonomic annotation of BGCs**

For this we will use the [MMseqs taxonomy](https://github.com/soedinglab/MMseqs2), for which we have containerized and created the corresponding run script [run_mmseqs_taxonomy.sh](https://github.com/pereiramemo/bioprospecting/blob/main/run_scripts/run_mmseqs_taxonomy.sh).  
As a reference we will be using the [UniProtKB/Swiss-Prot](https://www.uniprot.org/uniprotkb?facets=reviewed%3Atrue&query=%2A). 

Before doing anything, we have to organize the data and select the scaffolds in which a BGC was annotated:

In [21]:
%%bash

# mkdir "${WORKDIR}"/outputs/bgc_taxa

cat "${WORKDIR}"/outputs/bgc_abund/ERR*.tsv | cut -f1 | sort | uniq > "${WORKDIR}"/outputs/bgc_taxa/ids.txt
cat "${WORKDIR}"/data/sola/ERR*/output/scaffolds.fasta >  "${WORKDIR}"/outputs/bgc_taxa/all.fasta

"${seqtk}" subseq \
"${WORKDIR}"/outputs/bgc_taxa/all.fasta \
"${WORKDIR}"/outputs/bgc_taxa/ids.txt > \
"${WORKDIR}"/outputs/bgc_taxa/bgc.fasta


Now we can perform the taxonomic annotation:

In [22]:
%%bash

"${REPO}"/run_scripts/run_mmseqs_taxonomy.sh \
"${WORKDIR}"/outputs/bgc_taxa/bgc.fasta \
"${WORKDIR}"/outputs/bgc_taxa/bgc_taxa_annot \
--threads 40 \
--tax-lineage 1 \
-v 0


In [23]:
%%bash

head "${WORKDIR}"/outputs/bgc_taxa/bgc_taxa_annot_lca.tsv

ERR2604090__k119_185408	1890424	order	Synechococcales	26	25	18	0.710	-_cellular organisms;d_Bacteria;-_Terrabacteria group;-_Cyanobacteriota/Melainabacteria group;p_Cyanobacteriota;c_Cyanophyceae;o_Synechococcales
ERR2604090__k119_153928	2	superkingdom	Bacteria	9	8	6	0.750	-_cellular organisms;d_Bacteria
ERR2604090__k119_123795	31989	family	Paracoccaceae	4	4	2	0.500	-_cellular organisms;d_Bacteria;p_Pseudomonadota;c_Alphaproteobacteria;o_Rhodobacterales;f_Paracoccaceae
ERR2604090__k119_131898	1236	class	Gammaproteobacteria	7	7	5	0.800	-_cellular organisms;d_Bacteria;p_Pseudomonadota;c_Gammaproteobacteria
ERR2604090__k119_155472	1224	phylum	Pseudomonadota	10	10	7	0.670	-_cellular organisms;d_Bacteria;p_Pseudomonadota
ERR2604090__k119_52769	131567	no rank	cellular organisms	3	3	3	1.000	-_cellular organisms
ERR2604090__k119_148185	2	superkingdom	Bacteria	5	5	5	1.000	-_cellular organisms;d_Bacteria
ERR2604090__k119_84479	1224	phylum	Pseudomonadota	9	9	8	0.880	-_cellular organisms;d_Bacteri

<a id='section4'></a>
**4. Mapping BGCs onto reference Gene Cluster Families (GCFs)**

We will use [BiG-SLICE](https://github.com/medema-group/bigslice) to map the identified BGCs onto the GCFs of [BiG-FAM database](https://bigfam.bioinformatics.nl/).  
This tool requires the [dataset.tsv and taxonomy files](https://github.com/medema-group/bigslice/wiki/Input-folder). First the `dataset.tsv` with the following commands:  

In [28]:
%%bash

ls -d "${WORKDIR}/outputs/antismash/"ERR* | \
while read LINE; do

  DATASET=$(basename $(ls -d ${LINE}))
  PATH2DATASET=$(basename $(dirname ${LINE}))"/"
  echo -e "${DATASET}\t./\ttaxonomy/${DATASET}_taxonomy.tsv\tdataset_${DATASET}"

done > "${WORKDIR}/outputs/antismash/datasets.tsv"


Then, the `taxonomy.tsv` files utilizing our auxiliary script [create_taxonomy.py](https://github.com/pereiramemo/bioprospecting/blob/main/aux_scripts/create_taxonomy.py)

In [27]:
%%bash

"${REPO}"/aux_scripts/create_taxonomy.py \
--input_tsv "${WORKDIR}"/outputs/bgc_taxa/bgc_taxa_annot_lca.tsv \
--output_dir "${WORKDIR}/outputs/antismash/taxonomy"


Now let's download the [BiG-FAM database](https://bigfam.bioinformatics.nl/) (this can take some time)

In [9]:
%%bash

# wget http://bioinformatics.nl/~kauts001/ltr/bigslice/paper_data/data/full_run_result.zip --directory-prefix  ${WORKDIR}/data/
# unzip ${WORKDIR}/data/full_run_result.zip

And run [BiG-SLICE](https://github.com/medema-group/bigslice) using our containerized version:

In [30]:
%%bash

"${REPO}"/run_scripts/run_bigslice.sh query \
"${WORKDIR}/outputs/antismash/" \
"${WORKDIR}/data/full_run_result" \
--num_threads 40 \
--threshold_pct 0.1 \
--query_name SOLA


pid 103's current affinity list: 0-47
pid 103's new affinity list: 47
pid 104's current affinity list: 0-47
pid 104's new affinity list: 46
pid 105's current affinity list: 0-47
pid 105's new affinity list: 45
pid 106's current affinity list: 0-47
pid 106's new affinity list: 44
pid 107's current affinity list: 0-47
pid 107's new affinity list: 43
pid 108's current affinity list: 0-47
pid 108's new affinity list: 42
pid 109's current affinity list: 0-47
pid 109's new affinity list: 41
pid 110's current affinity list: 0-47
pid 110's new affinity list: 40
pid 111's current affinity list: 0-47
pid 111's new affinity list: 39
pid 112's current affinity list: 0-47
pid 112's new affinity list: 38
pid 113's current affinity list: 0-47
pid 113's new affinity list: 37
pid 114's current affinity list: 0-47
pid 114's new affinity list: 36
pid 115's current affinity list: 0-47
pid 115's new affinity list: 35
pid 116's current affinity list: 0-47
pid 116's new affinity list: 34
pid 117's current af

We can see the results in the folder `"${WORKDIR}/data/full_run_result"`

In [31]:
%%bash

ls "${WORKDIR}/data/full_run_result/reports"

1
reports.db


The main result we obtain are the SQLite databases. Although we could access these utilizing the mini web application based on Flask library, we are going to import them into an R environment to have full control of results.

In [39]:
%%R

library(RSQLite)
library(tidyverse)

conn_reports_db <- dbConnect(SQLite(), "workdir/data/full_run_result/reports/1/data.db")
conn_data_db <- dbConnect(SQLite(), "workdir/data/full_run_result/result/data.db")

In [40]:
%%R

dbListTables(conn_reports_db)

 [1] "bgc"             "bgc_class"       "bgc_features"    "cds"            
 [5] "gcf_membership"  "hsp"             "hsp_alignment"   "hsp_subpfam"    
 [9] "schema"          "sqlite_sequence"


In [46]:
%%R

gcf_membership_df <- dbReadTable(conn_reports_db, "gcf_membership")
head(gcf_membership_df)

  gcf_id bgc_id membership_value rank
1 244878    378         199.3951    0
2 244878    671         199.4237    0
3 244878    510         199.3276    0
4 244878    201         199.3147    0
5 244878    140         199.3254    0
6 244878    306         199.2584    0


In [47]:
%%R
gcf_membership_df$gcf_id %>% unique()

[1] 244878


In [56]:
%%R
dbListTables(conn_data_db)

 [1] "bgc"               "bgc_class"         "bgc_features"     
 [4] "bgc_taxonomy"      "cds"               "chem_class"       
 [7] "chem_subclass"     "chem_subclass_map" "clustering"       
[10] "dataset"           "enum_bgc_type"     "enum_run_status"  
[13] "gcf"               "gcf_membership"    "gcf_models"       
[16] "hmm"               "hmm_db"            "hsp"              
[19] "hsp_alignment"     "hsp_subpfam"       "run"              
[22] "run_bgc_status"    "run_log"           "schema"           
[25] "sqlite_sequence"   "subpfam"           "taxon"            
[28] "taxon_class"      


In [111]:
%%R
dbGetQuery(conn_data_db, "SELECT * FROM dataset LIMIT 10")

   id                       name                   orig_folder
1   1                      mibig                     mibig_2.0
2   2           isolate_archaeal          ncbi_archaea_genbank
3   3 isolate_bacterial_complete ncbi_bacteria_refseq_complete
4   4    isolate_bacterial_draft    ncbi_bacteria_refseq_draft
5   5             isolate_fungal            ncbi_fungi_genbank
6   6               mag_humangut                  almeida_2019
7   7                mag_chicken              glendinning_2020
8   8                    mag_uba                    parks_2017
9   9                 mag_bovine                  stewart_2019
10 10                  mag_ocean                    tully_2018
                                                                                           description
1                                                                             MIBiG 2.0 reference BGCs
2                                          BGCs from Archaeal genbanks taken at 2020-02-12 03:22 UTC+

If the dataset is not too large, we can use [BiG-SCAPE](https://github.com/medema-group/BiG-SCAPE) (instead of [BiG-SLICE](https://github.com/medema-group/bigslice)). Although this tool does not scale well, [it has a greater sensitivity compared to BiG-SLICE](https://academic.oup.com/gigascience/article/10/1/giaa154/6092777).

In [4]:
%%bash
ls "${REPO}"/run_scripts/run_bigscape.sh . . --help

Usage: ls [OPTION]... [FILE]...
List information about the FILEs (the current directory by default).
Sort entries alphabetically if none of -cftuvSUX nor --sort is specified.

Mandatory arguments to long options are mandatory for short options too.
  -a, --all                  do not ignore entries starting with .
  -A, --almost-all           do not list implied . and ..
      --author               with -l, print the author of each file
  -b, --escape               print C-style escapes for nongraphic characters
      --block-size=SIZE      with -l, scale sizes by SIZE when printing them;
                               e.g., '--block-size=M'; see SIZE format below
  -B, --ignore-backups       do not list implied entries ending with ~
  -c                         with -lt: sort by, and show, ctime (time of last
                               modification of file status information);
                               with -l: show ctime and sort by name;
                               othe

In [7]:
%%bash
# ls "${WORKDIR}"/outputs/antismash/ERR2604*/scaffolds/ERR*.region*.gbk | \
# while read LINE; do 

#   NEW_FILE_NAME=$(basename "${LINE/.gbk/_formatted.gbk}")
#   OLD_FILE_NAME=$(basename "${LINE/.gbk/_old.gbk}")
#   DIR_NAME=$(dirname "${LINE}")  
   
#   awk '{ if (NR == 1){print $1,$2} else {print $0}}' \
#   "${LINE}" > "${DIR_NAME}"/"${NEW_FILE_NAME}"
  
#   mv "${LINE}" "${DIR_NAME}"/"${OLD_FILE_NAME}"

# done  
        

In [11]:
%%bash

"${REPO}"/run_scripts/run_bigscape.sh \
"${WORKDIR}/outputs/antismash/" \
"${WORKDIR}/outputs/bigscape_clust/" \
--cores 40 > "${WORKDIR}"/outputs/bigscape.log


/usr/local/envs/bigscape/lib/python3.6/site-packages/Bio/GenBank/Scanner.py:1354: BiopythonParserWarning: Attempting to parse malformed locus line:
'LOCUS       ERR2604088__k119_116291 18303 bp    DNA     linear   UNK 01-JAN-1980\n'
Found locus 'ERR2604088__k119_116291' size '18303' residue_type 'DNA'
Some fields may be wrong.
  BiopythonParserWarning)
/usr/local/envs/bigscape/lib/python3.6/site-packages/Bio/GenBank/Scanner.py:1354: BiopythonParserWarning: Attempting to parse malformed locus line:
'LOCUS       ERR2604088__k119_111249 11770 bp    DNA     linear   UNK 01-JAN-1980\n'
Found locus 'ERR2604088__k119_111249' size '11770' residue_type 'DNA'
Some fields may be wrong.
  BiopythonParserWarning)
/usr/local/envs/bigscape/lib/python3.6/site-packages/Bio/GenBank/Scanner.py:1354: BiopythonParserWarning: Attempting to parse malformed locus line:
'LOCUS       ERR2604088__k119_102369 19680 bp    DNA     linear   UNK 01-JAN-1980\n'
Found locus 'ERR2604088__k119_102369' size '19680' residu